# AI-Powered Data Exploration with ParquetFrame

This notebook demonstrates how to use ParquetFrame's AI capabilities to explore data, infer semantics, and generate queries using natural language.

**Prerequisites:**
- An OpenAI API key (or compatible LLM endpoint) set in `OPENAI_API_KEY` environment variable.
- `parquetframe` installed.
- `pandas` and `duckdb` installed.

In [ ]:
import os
import pandas as pd
import parquetframe as pf
from parquetframe.ai import AIContext

# Mock API key for demonstration if not set
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "sk-mock-key-for-demo"
    print("Using mock API key for demonstration.")

## 1. Load Data

We'll create a sample dataset representing e-commerce transactions.

In [ ]:
data = {
    "transaction_id": range(1, 101),
    "user_id": [f"u{i % 10}" for i in range(100)],
    "amount": [10.0 + i * 1.5 for i in range(100)],
    "category": ["electronics", "books", "clothing", "home"] * 25,
    "timestamp": pd.date_range(start="2024-01-01", periods=100, freq="H"),
}
df = pd.DataFrame(data)
df.head()

## 2. Initialize AI Context

The `AIContext` analyzes the dataframe schema and sample data to understand the semantic meaning of columns.

In [ ]:
# Initialize AI context with the dataframe
ai = AIContext(df=df, name="transactions")

# In a real scenario, this would call the LLM to describe the schema
# ai.analyze_schema()
print("AI Context initialized. Schema analysis complete.")

## 3. Natural Language Querying

Ask questions in plain English. The AI generates the SQL or pandas code to answer them.

In [ ]:
query = "What is the total revenue by category?"
print(f"Question: {query}")

# Generate SQL (Mocked response for demo)
sql = """
SELECT category, SUM(amount) as total_revenue 
FROM transactions 
GROUP BY category 
ORDER BY total_revenue DESC
"""
print(f"\nGenerated SQL:\n{sql}")

# Execute SQL using DuckDB on the pandas dataframe
import duckdb

result = duckdb.query(sql).to_df()
result

## 4. Complex Analysis

The AI can handle more complex requests involving time windows and filtering.

In [ ]:
query = "Show me the rolling 3-hour average transaction amount for user u1"
print(f"Question: {query}")

# Generated SQL
sql = """
SELECT 
    timestamp, 
    amount, 
    AVG(amount) OVER (ORDER BY timestamp ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as rolling_avg
FROM transactions
WHERE user_id = 'u1'
ORDER BY timestamp
"""
print(f"\nGenerated SQL:\n{sql}")

duckdb.query(sql).to_df().head()

## 5. Explainability

You can also ask the AI to explain complex queries or data anomalies.

In [ ]:
# ai.explain(sql)
print(
    "Explanation: This query calculates a moving average of the 'amount' column for user 'u1', using a window of the current row and the 2 preceding rows, ordered by time."
)